# FileTree
```
Dependencies: treelib, os
```
Extension of treelib to provide handy utilities for making directories with ML/data science experiments in mind.
Two ways of creating trees
1. 'node view' this has finer control of appending to the tree (termed *unique node tree*)
1. 'product view' this includes list of lists - each list is created in sublevel of element of higher level list (termed *product tree*). For example if inputting a list of lists `[[a,b],[C,D]]` we would expect the following tree 

```
root
├── a
│   ├── C
│   └── D
└── b
    ├── C
    └── D
```

The latter may for example capture different values of tuning parameters that are then organised into a tree hierarchy.

The package can create directories to disk after designing the tree.
It is then possible to filter paths based on keywords.
The user may wish to, for example, organise higher level directories according to a timestamp.

#### Additional details
Tree data structure recap:
1. Each node is assigned a unique `identifier`. Used when adding a child node to a parent.
1. Node data -- in this case carrying the name of a folder -- is stored in the node `tag` property

See https://treelib.readthedocs.io/en/latest/ for more details.

#### FileTree

In [9]:
from FileTree import FileTree

# Examples

### Simple node tree
Get the path to a node through specifying the node name. At the moment, the tag (`create_node`'s first argument must be the same as the identifier). `path_from_root` then prints the list of tags that connects the specified node with the root

In [2]:

tree = FileTree()
n1 = tree.create_node('node1',identifier='1')
n2 = tree.create_node('node2',identifier='2', parent='1')
n3 = tree.create_node('node3',identifier='3', parent='1')
node = tree.create_node('node4',identifier='4',parent='3')

tree.show()
print(tree.path_search('node2'),'\n')
print(tree.path_search('node4'),'\n')


node1
├── node2
└── node3
    └── node4

['node1/node2'] 

['node1/node3/node4'] 



### Product trees
Make a product tree through list of lists

In [3]:

product_tree = FileTree()
root_node = product_tree.create_node('root')

product_lists = (['a','b'],['1','2'],['alpha','beta','gamma'])
product_tree.append_product_tree(root_node,*product_lists)
product_tree.show()


root
├── a
│   ├── 1
│   │   ├── alpha
│   │   ├── beta
│   │   └── gamma
│   └── 2
│       ├── alpha
│       ├── beta
│       └── gamma
└── b
    ├── 1
    │   ├── alpha
    │   ├── beta
    │   └── gamma
    └── 2
        ├── alpha
        ├── beta
        └── gamma



### Append a product tree to a manually created tree
Paste the product tree into the unique node tree (this may lose uniqueness)

In [4]:
appending_node = tree.get_node('2')

tree.append_product_tree(appending_node, *product_lists)
tree.show()

node1
├── node2
│   ├── a
│   │   ├── 1
│   │   │   ├── alpha
│   │   │   ├── beta
│   │   │   └── gamma
│   │   └── 2
│   │       ├── alpha
│   │       ├── beta
│   │       └── gamma
│   └── b
│       ├── 1
│       │   ├── alpha
│       │   ├── beta
│       │   └── gamma
│       └── 2
│           ├── alpha
│           ├── beta
│           └── gamma
└── node3
    └── node4



### Make directories in current location
Calls the function to make all the directories based on the previously combined tree

In [5]:
tree.show()
tree.make_dirs()

node1
├── node2
│   ├── a
│   │   ├── 1
│   │   │   ├── alpha
│   │   │   ├── beta
│   │   │   └── gamma
│   │   └── 2
│   │       ├── alpha
│   │       ├── beta
│   │       └── gamma
│   └── b
│       ├── 1
│       │   ├── alpha
│       │   ├── beta
│       │   └── gamma
│       └── 2
│           ├── alpha
│           ├── beta
│           └── gamma
└── node3
    └── node4


written directory tree -- (exist_ok=True)


# Data science example
Example used to generate directory structure in intro.

In [6]:
# initalise tree
tree = FileTree()

# make unique node tree
tree.create_node('project_name',identifier='project_name')
tree.create_node('time_stamp_1',identifier='time_stamp_1',parent='project_name')
tree.create_node('time_stamp_2',identifier='time_stamp_2',parent='project_name')

# define list of lists that will be used to make product trees
product_lists = [
    ['data_set_1','data_set_2'],
    ['param_val_1','param_val_2','param_val_3'],
]

# get the first node to append the product tree to
appending_node = tree.get_node('time_stamp_1')
tree.append_product_tree(appending_node, *product_lists)

# get the second node to append the product tree to
appending_node = tree.get_node('time_stamp_2')
tree.append_product_tree(appending_node, *product_lists)
tree.show()


project_name
├── time_stamp_1
│   ├── data_set_1
│   │   ├── param_val_1
│   │   ├── param_val_2
│   │   └── param_val_3
│   └── data_set_2
│       ├── param_val_1
│       ├── param_val_2
│       └── param_val_3
└── time_stamp_2
    ├── data_set_1
    │   ├── param_val_1
    │   ├── param_val_2
    │   └── param_val_3
    └── data_set_2
        ├── param_val_1
        ├── param_val_2
        └── param_val_3



### Search for paths
Also able to search the tree for nodes with a particular name.
By default, the filter asserts all path exist on disk.

In [7]:
# tree.path_search(lambda x: x.tag=='param_val_1' | x.is_ancestor('time_stamp_1'))

tree.show()

keywords = ['time_stamp_1','param_val_1']
print('search tree for keywords: {}'.format(keywords))

print('\nRequire all search terms appear in path to root')
search_res = tree.path_search(keywords,req_all_keywords=True)
for s in search_res:
    print(s)

print('\n\nRequire any of the search terms appear in path to root')   
search_res = tree.path_search(keywords,req_all_keywords=False)
for s in search_res:
    print(s)


project_name
├── time_stamp_1
│   ├── data_set_1
│   │   ├── param_val_1
│   │   ├── param_val_2
│   │   └── param_val_3
│   └── data_set_2
│       ├── param_val_1
│       ├── param_val_2
│       └── param_val_3
└── time_stamp_2
    ├── data_set_1
    │   ├── param_val_1
    │   ├── param_val_2
    │   └── param_val_3
    └── data_set_2
        ├── param_val_1
        ├── param_val_2
        └── param_val_3

search tree for keywords: ['time_stamp_1', 'param_val_1']

Require all search terms appear in path to root
project_name/time_stamp_1/data_set_1/param_val_1
project_name/time_stamp_1/data_set_2/param_val_1


Require any of the search terms appear in path to root
project_name/time_stamp_1
project_name/time_stamp_1/data_set_1/param_val_1
project_name/time_stamp_1/data_set_2/param_val_1
project_name/time_stamp_2/data_set_1/param_val_1
project_name/time_stamp_2/data_set_2/param_val_1
